<a href="https://colab.research.google.com/github/rafik-chemli/NRCan-Statcan-Furnace-CFD-Analysis/blob/main/NRCan_Statcan_CFD_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive # Ignore If not using google drive
drive.mount('/content/drive')

Mounted at /content/drive


## PreProcessing
This code processes multiple CSV files containing particle information in a target folder, calculates relevant physical quantities, determines the state of particles based on these calculations, and aggregates the results in a static format.

In [ ]:
%%time
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import re

# Takes ~ 25mins for 10 files of ~1go

folder_path = '/content/drive/MyDrive/Colab Notebooks/statcan/furnace/CFD_particle_data'
rho_L = 3440
gamma = 0.568

def sanitize_column_names(dataframe):
    """
    Sanitize column names by replacing spaces, periods, slashes, and dashes with underscores.
    :param dataframe: DataFrame with column names to sanitize.
    :return: DataFrame with sanitized column names.
    """
    sanitized_columns = dataframe.columns.str.strip().str.replace(' ', '_').str.replace('.', '').str.replace('/', '_').str.replace('-', '')
    dataframe.columns = sanitized_columns
    return dataframe

def calculate_we_bo_lambda(row, rho_L, gamma):
    """
    Calculate Weber number, Bond number, and lambda squared for a particle.
    :param row: Series representing a particle.
    :param rho_L: Liquid density.
    :param gamma: Surface tension coefficient.
    :return: Tuple of Weber number, Bond number, and lambda squared.
    """
    velocity_components = [row['ParticleXVelocity_m_s'], row['ParticleYVelocity_m_s'], row['ParticleZVelocity_m_s']]
    velocity_magnitude = np.sqrt(sum(v**2 for v in velocity_components))
    We = rho_L * velocity_magnitude**2 * row['ParticleDiameter_m'] / gamma
    Bo = rho_L * 9.81 * row['ParticleDiameter_m']**2 / gamma
    lambda_squared = (row['ParticleDensity_kg_m3'] / rho_L) ** 2
    return We, Bo, lambda_squared

def determine_state(row, rho_L, gamma):
    """
    Determine the state based on calculated Weber, Bond numbers, and lambda squared.
    :param row: Series representing a particle.
    :param rho_L: Liquid density.
    :param gamma: Surface tension coefficient.
    :return: Integer state value.
    """
    We, Bo, lambda_squared = calculate_we_bo_lambda(row, rho_L, gamma)
    WeBo = We * np.sqrt(Bo**3)
    return 0 if WeBo >= 12 / lambda_squared else 1 if 6 / lambda_squared <= WeBo < 12 / lambda_squared else 2

def extract_info_from_filename(filename):
    """
    Extract bar, kgm3, and VM values from the filename. Ignore files not starting with 'file'
    and those prefixed with 'processed'.
    :param filename: The filename to extract data from.
    :return: Tuple of extracted values or (None, None, None) if not valid, processed, or not starting with 'file'.
    """
    # if not filename.startswith('file') or filename.startswith('processed'):
    #     return None, None, None

    pattern = r'file_(\d+\.?\d*)bar_(\d+\.?\d*)kgm3_(\d+\.?\d*)VM\.csv'
    match = re.search(pattern, filename)
    return match.groups() if match else (None, None, None)


def process_file(df, rho_L, gamma, start_tracker_id):
    """
    Process each file by determining state, adjusting time, and assigning tracker IDs.
    The tracker IDs continue incrementing from the last ID used in the previous file.
    :param df: DataFrame of particle data.
    :param rho_L: Liquid density.
    :param gamma: Surface tension coefficient.
    :param start_tracker_id: The starting tracker ID to ensure continuity across files.
    :return: DataFrame after processing and the last tracker ID used.
    """

    df['AdjustedTime'] = df.groupby('ParticleID_').apply(
        lambda x: x['ParticleResidenceTime_s'] + x.groupby((x['ParticleResidenceTime_s'].diff() > 0).cumsum()).cumcount() * 1e-11
    ).reset_index(level=0, drop=True)
    processed_df, last_tracker_id = assign_tracker_ids(df, start_tracker_id)
    return processed_df, last_tracker_id

def assign_tracker_ids(df, start_tracker_id):
    """
    Assign tracker IDs based on Particle ID and Adjusted Time, ensuring global uniqueness.
    Tracker IDs increment with each new ParticleID or when the AdjustedTime decreases (reset in time).
    :param df: DataFrame with particle data.
    :param start_tracker_id: The starting tracker ID to ensure uniqueness.
    :return: DataFrame with tracker IDs assigned and the last used tracker_id.
    """
    current_particle_id = None
    previous_time = None
    tracker_id = start_tracker_id
    tracker_ids = []

    for _, row in df.iterrows():
        if row['ParticleID_'] != current_particle_id:
            current_particle_id = row['ParticleID_']
            tracker_id += 1
            previous_time = None
        elif previous_time is not None and row['AdjustedTime'] < previous_time:
            tracker_id += 1
        tracker_ids.append(tracker_id)
        previous_time = row['AdjustedTime']

    df['tracker_id'] = tracker_ids
    return df, tracker_id

desired_columns = [
    'ParticleID_', 'ParticleXPosition_m', 'ParticleYPosition_m', 'ParticleZPosition_m',
    'ParticleXVelocity_m_s', 'ParticleYVelocity_m_s', 'ParticleZVelocity_m_s',
    'ParticleDiameter_m', 'ParticleTemperature_K', 'ParticleDensity_kg_m3',
    'ParticleMass_kg', 'ParticleRadialPosition_m', 'ParticleThetaPosition',
    'ParticleRadialVelocity_m_s', 'ParticleSwirlVelocity_m_s',
    'ParticleVelocityMagnitude_m_s', 'ParticleSpecificHeat',
    'ParticleBinaryDiffusivity', 'ParticleLawIndex', 'ParticleReynoldsNumber',
    'ParticleLiquidVolumeFraction', 'ParticleLiquidMassFraction',
    'ParticleVolatileMassFraction', 'ParticleCharMassFraction',
    'VaporizationLimitingTime_s', 'ParticleLewisNumber', 'ParticleNusseltNumber',
    'BMMassTransferNumber', 'BMHeatTransferNumber', 'state',
    'pressure_bar', 'density_kgm3', 'volatile_matter_VM', 'tracker_id'
]


def process_all_files(folder_path, rho_L, gamma):
    """
    Process all files in the specified folder, ensuring that tracker IDs are unique globally.
    :param folder_path: Path to the folder containing the files.
    :param rho_L: Liquid density.
    :param gamma: Surface tension coefficient.
    :return: DataFrame with all processed data.
    """
    all_data = pd.DataFrame()
    global_tracker_id = 0  # Initialize global tracker ID
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('file')]
    for filename in tqdm(file_list, desc='Processing files'):
        pressure, density, volatile_matter = extract_info_from_filename(filename)
        if all([pressure, density, volatile_matter]):
            df_path = os.path.join(folder_path, filename)
            data = pd.read_csv(df_path)
            data = sanitize_column_names(data)
            processed_data, last_tracker_id = process_file(data, rho_L, gamma, global_tracker_id)
            final_data = processed_data.drop_duplicates('tracker_id', keep='last').reset_index(drop=True)
            final_data['pressure_bar'] = pressure
            final_data['density_kgm3'] = density
            final_data['volatile_matter_VM'] = volatile_matter
            final_data['state'] = final_data.apply(determine_state, args=(rho_L, gamma), axis=1)
            final_data = final_data.loc[:, final_data.columns.intersection(desired_columns)]
            all_data = pd.concat([all_data, final_data], ignore_index=True)
            global_tracker_id = last_tracker_id
        else:
            print(f'{filename} is not an expected file name and will not be processed')
    return all_data

# Process all files and save the aggregated data

all_data = process_all_files(folder_path, rho_L, gamma)
output_path = os.path.join(folder_path, "aggregated_processed_data.csv")
all_data.to_csv(output_path, index=False)
print(f"All data processed and saved: {output_path}")


## Interactive Visualization
Parallel Coordinates plot to enable the exploration of multi-dimensional data

In [1]:
import plotly.express as px
import pandas as pd

all_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/statcan/furnace/CFD_particle_data/aggregated_processed_data.csv')

# Ensure the specified columns are treated as floats
all_data['ParticleDiameter_m'] = all_data['ParticleDiameter_m'].astype(float)
all_data['ParticleMass_kg'] = all_data['ParticleMass_kg'].astype(float)
all_data['pressure_bar'] = all_data['pressure_bar'].astype(float)
all_data['density_kgm3'] = all_data['density_kgm3'].astype(float)
all_data['volatile_matter_VM'] = all_data['volatile_matter_VM'].astype(float)
all_data['state'] = all_data['state'].astype(str)

# Create the Parallel Coordinates plot with custom color mapping for 'state'
fig = px.parallel_coordinates(all_data,
                              dimensions=[
                                  'ParticleDiameter_m',
                                  'ParticleMass_kg',
                                  'pressure_bar',
                                  'density_kgm3',
                                  'volatile_matter_VM',
                                  'state'
                              ],
                              labels={
                                  'ParticleDiameter_m': 'Particle Diameter (m)',
                                  'ParticleMass_kg': 'Particle Mass (kg)',
                                  'pressure_bar': 'Pressure (bar)',
                                  'density_kgm3': 'Density (kg/m³)',
                                  'volatile_matter_VM': 'Velocity Multiplier (VM)',
                                  'state': 'State'
                              })

fig.show()



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/statcan/furnace/CFD_particle_data/aggregated_processed_data.csv'

##  Modelisation
Optimizing and evaluating different machine learning models to predict the state of particles in CFD simulations. Utilizing a combination of RandomForest, GradientBoosting, and XGBoost classifiers

In [ ]:
%%time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import joblib
from xgboost import XGBClassifier
import numpy as np

# Remove duplicates from the dataset
all_data = all_data[['ParticleDiameter_m', 'ParticleMass_kg', 'pressure_bar', 'density_kgm3', 'volatile_matter_VM', 'state']].drop_duplicates()

# Prepare dataset columns
X = all_data[['ParticleDiameter_m', 'ParticleMass_kg', 'pressure_bar', 'density_kgm3', 'volatile_matter_VM']]
y = all_data['state'].astype(int)  # Ensure y is of integer type

# Split the dataset into training, validation, and test sets with stratification
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Define the pipeline and parameter grid for each model
models_params = {
    'random_forest': {
        'pipeline': Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', RandomForestClassifier(random_state=42))
        ]),
        'params': {
            'classifier__n_estimators': [100, 200],
            'classifier__max_depth': [10, 20, None]
        }
    },
    'gradient_boosting': {
        'pipeline': Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', GradientBoostingClassifier(random_state=42))
        ]),
        'params': {
            'classifier__n_estimators': [100, 200],
            'classifier__learning_rate': [0.05, 0.1],
            'classifier__max_depth': [3, 5, 10]
        }
    },
    'xgboost': {
        'pipeline': Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
        ]),
        'params': {
            'classifier__n_estimators': [100, 200],
            'classifier__learning_rate': [0.05, 0.1],
            'classifier__max_depth': [3, 5, 7]
        }
    }
}

# Initialize the dictionary to store optimized results
optimized_results = {}

# Iterate through each model configuration
for model_name, model_info in models_params.items():
    # Perform grid search using the training data
    grid_search = GridSearchCV(model_info['pipeline'], model_info['params'], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Get the best model from grid search
    best_model = grid_search.best_estimator_

    # Evaluate the best model on the test set
    y_test_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_report = classification_report(y_test, y_test_pred)

    # Evaluate the same model on the validation set
    y_val_pred = best_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_report = classification_report(y_val, y_val_pred)

    # Store results including the validation performance
    optimized_results[model_name] = {
        'best_model': best_model,
        'best_params': grid_search.best_params_,
        'test_accuracy': test_accuracy,
        'test_classification_report': test_report,
        'validation_accuracy': val_accuracy,
        'validation_classification_report': val_report
    }

optimized_results


### Classification report of the best performing model trained

In [ ]:
#no free lunch theorem
# Choose the best model based on accuracy
best_model_name = max(optimized_results, key=lambda x: optimized_results[x]['validation_accuracy'])
best_model_details = optimized_results[best_model_name]

# Save the best model details, including the pipeline
filepath = os.path.join(folder_path, "best_model.pkl")
joblib.dump(best_model_details, filepath)

# Print relevant information
print(best_model_name)
print(best_model_details['validation_classification_report'])
print("State Mapping: {0: 'Penetrating', 1: 'Bouncing', 2: 'Oscillating'}")
# Load the model details back
loaded_model_details = joblib.load(filepath)

xgboost
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       989
           1       0.71      0.80      0.75        86
           2       0.99      1.00      0.99      1953

    accuracy                           0.98      3028
   macro avg       0.90      0.93      0.91      3028
weighted avg       0.99      0.98      0.98      3028

State Mapping: {0: 'Penetrating', 1: 'Bouncing', 2: 'Oscillating'}


# Predict the fate of the particle based on input data
(Assuming the model is already generated in the target folder)

In [ ]:
import joblib
import os
import pandas as pd
import numpy as np

folder_path = '/content/drive/MyDrive/Colab Notebooks/statcan/furnace/CFD_particle_data'
filename = "best_model.pkl"
filepath = os.path.join(folder_path, filename)

loaded_model_details = joblib.load(filepath)

# Loading the model with the best parameters:
loaded_model = loaded_model_details['best_model']
# You can now use loaded_model for predictions or further analysis.

# Example input values for multiple particles
input_data = {
    'ParticleDiameter_m': [0.005, 0.007, 0.004],  # Example diameters for three particles
    'ParticleMass_kg':    [0.01, 0.02, 0.015],    # Corresponding masses for those particles
    'pressure_bar':       [100, 100, 100],
    'density_kgm3':       [500, 500, 500],
    'volatile_matter_VM': [50, 50, 50],
}

input_df = pd.DataFrame(input_data)

# Assuming 'loaded_model' is your trained model loaded from the previous step
predicted_states = loaded_model.predict(input_df)

# Dictionary mapping state codes to descriptions
state_descriptions = {0: 'Penetrating', 1: 'Bouncing', 2: 'Oscillating'}

# Print predictions for each particle
for i, state in enumerate(predicted_states):
    print(f"Particle {i+1} Prediction: {state} ({state_descriptions[state]})")


Particle 1 Prediction: 0 (Penetrating)
Particle 2 Prediction: 0 (Penetrating)
Particle 3 Prediction: 0 (Penetrating)
